In [ ]:
from functools import reduce
import hashlib
import struct

import base58
from factom import Factomd
from factom_core.blocks import DirectoryBlock, FactoidBlock

## Functions that are translated to Solidity

These functions contain the smart contract logic, which essentially takes the raw data for a Factoid transaction, such as tx input, tx output, input & output counts and amounts, as well as the elements along the Merkle authentication path for the transaction, necessary to reconstruct the Factom anchor recorded on Ethereum.

It proceeds by:
* computing the transaction hash
* asserting that the transaction satisfies (an imaginary) request for a given amount of FCT to be transferred to a given address
* computing the window Merkle root, recorded on Ethereum, from the transaction hash and the Merkle authenation path elements

In [ ]:
def varint_encode(number: int):
    """Pack `number` into varint bytes"""
    buf = bytearray()
    if number == 0:
        buf.append(0x00)

    h = number
    start = False
    if 0x8000000000000000 & h != 0:
        buf.append(0x81)
        start = True

    for i in range(9):
        b = h >> 56
        if b != 0 or start:
            start = True
            b = b | 0x80 if i != 8 else b & 0x7F
            b = b if b < 256 else struct.pack(">Q", b)[-1]
            buf.append(b)

        h = h << 7

    return bytes(buf)

def verify_merkle_path(elem, path, root, hash_f=hashlib.sha256):
    right = 0
    left = 1
    val1 = elem
    for val2, left_or_right in path:
        val1 = hash_f(val1 + val2).digest() if left_or_right is right else hash_f(val2 + val1).digest()
    return val1 == root

def derive_fct_address(address):
    address = b'\x5f\xb1' + address
    checksum = hashlib.sha256(hashlib.sha256(address).digest()).digest()[:4]
    return base58.b58encode(address + checksum)

# This is the main function that checks the validity of the transaction for fulfilling a request
def smart_contract_logic(
    tx_timestamp: bytes, 
    input_count: bytes,
    output_count: bytes,
    ec_count: bytes,
    input_addresses: list,
    input_amounts: list,
    output_address: bytes,
    output_amount: int,
    ec_amounts: list,
    ec_addresses: list,
    rcds: list,
    signatures: list,
    tx_block_height: int,
    tx_merkle_path: list,
    factoid_block_header_hash: bytes,
    factoid_block_body_merkle_root: bytes,
    factoid_block_merkle_path: list,
    directory_block_body_merkle_root: bytes,
    directory_block_header_hash: bytes,
    directory_block_merkle_path: list,
    fct_address_for_request: bytes,
    fct_amount_for_request: int,
):
    assert output_amount == fct_amount_for_request
    
    # Derive the human-readable FCT address from output_address
    fct_address = derive_fct_address(output_address)
    assert fct_address == fct_address_for_request
    
    buf = bytearray()
    buf.append(0x02)
    buf.extend(tx_timestamp)
    buf.extend(input_count)
    buf.extend(output_count)
    buf.extend(ec_count)
    for i in range(len(input_amounts)):
        buf.extend(varint_encode(input_amounts[i]) + input_addresses[i])
    buf.extend(varint_encode(output_amount) + output_address)
    for i in range(len(ec_amounts)):
        buf.extend(varint_encode(ec_amounts[i]) + ec_addresses[i])
    for i in range(len(input_amounts)):
        rcd = rcds[i]
        signature = signatures[i]
        buf.append(0x01)
        buf.extend(rcd)
        buf.extend(signature)
    
    tx_hash = hashlib.sha256(bytes(buf)).digest()    
    assert verify_merkle_path(tx_hash, tx_merkle_path, factoid_block_body_merkle_root)
    
    factoid_block_key_mr = hashlib.sha256(factoid_block_header_hash + factoid_block_body_merkle_root).digest()
    assert verify_merkle_path(
        factoid_block_key_mr,
        factoid_block_merkle_path,
        directory_block_body_merkle_root
    )
    
    directory_block_key_mr = hashlib.sha256(
        directory_block_header_hash + directory_block_body_merkle_root).digest()
    # In the smart contract, this should be replaced with a call to the contract with the anchors
    window_merkle_root = bytes.fromhex(
        factomd.anchors(height=tx_block_height)['ethereum']['windowmr']
    )
    
    assert verify_merkle_path(
        directory_block_key_mr,
        directory_block_merkle_path,
        window_merkle_root
    )

## Helper functions (not implemented in Solidity)

These functions are used to compute the elements in the Merkle authentication path for the given Factoid tx. They are not used inside the Ethereum smart contract (the outputs of these function serve as inputs to the contract).

In [ ]:
def compute_merkle_root(leaves, hash_f=hashlib.sha256):
    if len(leaves) == 1:
        return leaves[0]

    next_level = []

    for i in range(0, len(leaves), 2): 
        val_1 = leaves[i]
        val_2 = leaves[i+1] if i+1 != len(leaves) else leaves[i]
        next_level.append(hash_f(val_1 + val_2).digest())
        
    return compute_merkle_root(next_level)

def build_merkle_tree(leaves, hash_f=hashlib.sha256):
    if len(leaves[-1]) == 0 or len(leaves[-1]) == 1:
        return leaves

    next_level = []
    for i in range(0, len(leaves[-1]), 2):
        left = leaves[-1][i]
        right = leaves[-1][i + 1] if i + 1 != len(leaves[-1]) else left
        top = hash_f(left + right).digest()
        next_level.append(top)
    
    leaves.append(next_level)
    return build_merkle_tree(leaves)

def compute_merkle_path(index, tree, path=None):
    if len(tree) == 0 or len(tree) == 1:
        return path
    
    if path is None:
        path = []
    
    if index % 2 == 1:
        path.append((tree[0][index - 1], index % 2))
    else:
        if len(tree[0]) == index + 1:
            path.append((tree[0][index], index % 2))
        else:
            path.append((tree[0][index + 1], index % 2))
    return compute_merkle_path(index // 2, tree[1:], path)

def build_merkle_path_from_anchors(anchors):
    path = []
    left, right = 1, 0
    merkle_branch = anchors['ethereum']['merklebranch']
    current = anchors['directoryblockkeymr']
    for entry in merkle_branch:
        if entry['left'] == current:
            path.append((bytes.fromhex(entry['right']), right))
        else:
            path.append((bytes.fromhex(entry['left']), left))
        current = entry['top']
    return path

def factoid_block_body_leaves(factoid_block):
    leaves = []
    for transactions in factoid_block.body.transactions.values():
        for tx in transactions:
            leaves.append(tx.hash)
        minute_marker = hashlib.sha256(b"\x00").digest()
        leaves.append(minute_marker)
    return leaves


# Simulated execution of the contract

## Compute the smart contract inputs

In [ ]:
factomd = Factomd(host="https://api.factomd.net")
factomd.heights()

In [ ]:
fb = factomd.factoid_block_by_height(213715)
factoid_block = FactoidBlock.unmarshal(bytes.fromhex(fb["rawdata"]))
leaves = factoid_block_body_leaves(factoid_block)
fb_body_mt = build_merkle_tree([leaves])
fb_body_mr = compute_merkle_root(leaves)

In [ ]:
db = factomd.directory_block_by_height(213715)
db_entries = db['dblock']['dbentries']
leaves = []
for db_entry in db_entries:
    leaves.append(bytes.fromhex(db_entry['chainid']))
    leaves.append(bytes.fromhex(db_entry['keymr']))
db_mt = build_merkle_tree([leaves])

In [ ]:
directory_block = DirectoryBlock.unmarshal(bytes.fromhex(db['rawdata']))
anchors = factomd.anchors(directory_block.keymr)

## "Run" the smart contract

If no expception occurs when running `smart_contract_logic`, this means that the Factoid transaction has been successfully verified. 

In [ ]:
tx = factoid_block.body.transactions[1][1]
smart_contract_logic(
    tx_timestamp=tx.timestamp.to_bytes(6, 'big'),
    input_count=len(tx.inputs).to_bytes(1, 'big'),
    output_count=len(tx.outputs).to_bytes(1, 'big'),
    ec_count=len(tx.ec_purchases).to_bytes(1, 'big'),
    input_addresses=list(map(lambda x: x['fct_address'], tx.inputs)),
    input_amounts=list(map(lambda x: x['value'], tx.inputs)),
    output_address=list(map(lambda x: x['fct_address'], tx.outputs))[0],
    output_amount=list(map(lambda x: x['value'], tx.outputs))[0],
    ec_addresses=list(map(lambda x: x['ec_public_key'], tx.ec_purchases)),
    ec_amounts=list(map(lambda x: x['value'], tx.ec_purchases)),
    rcds=list(map(lambda x: x.public_key, tx.rcds)),
    signatures=list(map(lambda x: x.signature, tx.rcds)),
    fct_amount_for_request=list(map(lambda x: x['value'], tx.outputs))[0],
    fct_address_for_request=b'FA2akCVKkeKB7XLbxPU12eZnStMXXnqCR4uye4kCjmBNkXrnAC2H',
    tx_merkle_path=compute_merkle_path(fb_body_mt[0].index(tx.hash), fb_body_mt),
    factoid_block_body_merkle_root=factoid_block.body.merkle_root,
    factoid_block_header_hash=hashlib.sha256(factoid_block.header.marshal()).digest(),
    factoid_block_merkle_path=compute_merkle_path(db_mt[0].index(factoid_block.keymr), db_mt),
    directory_block_body_merkle_root=directory_block.header.body_mr,
    directory_block_header_hash=hashlib.sha256(directory_block.header.marshal()).digest(),
    directory_block_merkle_path=build_merkle_path_from_anchors(anchors),
    tx_block_height=factoid_block.header.height
)

# Description of Factoid block raw data

The information below is just for illustrative purposes and was used during the development of the Python logic
above. The raw data is for block number 213715 and can be accessed using `fb["rawdata"]`

In [ ]:
# # Factoid block header start
# '000000000000000000000000000000000000000000000000000000000000000f' -> FCT chain (32 bytes)
# 'd2dd9b16ea6a52e562e38607afd165d5a376ce10f248acb97f2f2c49b33f8f9c' -> body MR (32 bytes)
# 'a689e7c54c8bdb2e1025c8471dfd103038ade254101252e2d2bd773025bb8af0' -> Previous Key MR (32 bytes)
# '8494734948bcdac40516d66990a3e654a8d5e24d935e5a99db5551b822666299' -> Previous Ledger Key MR (32 bytes)
# '0000000000007148' -> Exchange rate (8 bytes)
# '000342d3' -> Block height (4 bytes)
# '00' -> Header Expansion Size (2 bytes)
# '00000002' -> Number of transactions (4 bytes)
# '00000150' -> Number of bytes in the block body (4 bytes)
# # Factoid block header end
# # Factoid block body start

# # Tx 1
# '02' -> Factoid tx marker (1 byte)
# '016db650053d' -> tx timestamp (6 bytes) (int.from_bytes(bytes.fromhex("016db650053d"), "big"))
# '00' -> input count (1 byte)
# '00' -> output count (1 byte)
# '00' -> entry credit count (1 byte)
# # End Tx1

# # Tx 2
# '02' -> Factoid tx marker (1 byte)
# '016db650c90a' -> tx timestamp (6 bytes)
# '02' -> input count (1 byte)
# '01' -> output count (1 byte)
# '00' -> ec count (1 byte)
# 'c4a9d38f1a' -> input amount (varint) varint_decode(bytes.fromhex('c4a9d38f1a'))
# '77fd94eacd8191bef2b42cef6108eaeae28d1674354e169e88a9e6f0eefa29e8' -> FCT input address (32 bytes)
# 'c3d38b8c0e' -> input amount (varint)
# '2d9723cd1d4f288ba4ab4f03974255f402913a8ebd7bb71b009481b08e566159' -> FCT input address
# '8187fcc79a00' -> output amount (varint)
# '50fc0b6f3cae9d7677bb1d84170cb595269216d0b381666999cf8d9663ccc637' -> FCT output address
# '01813f7e6a2d87d9918eae6866a5abe9fe2151a81a0cbd81f6f552898e5894c2fb' -> RCD 0
# '185b9985bddb3a38ea8c8841c304194068bfd208ae3fe36618633ed9e33e2b303ee8a7536f59f3894f72f075b8dbeb50dd288e39b64a524b8231c43c8303cb0c' -> signature 0
# '0151d3684db90b61283c08eabdd9d0660d43b5640f30787d0235dfc25c0a59afec' -> RCD 1
# 'bd566a110053fbd16d8c5b4ff7afb6c9e56551bbf95ff97206a72778e3495aa7979889c1cce941f663342ad501a794367ec0641155c4f4475b3e3a6509c09806' -> signature 1
# # Tx 2 end
# '00' -> minute 0 end
# '00' -> minute 1 end
# '00' -> minute 2 end
# '00' -> minute 3 end
# '00' -> minute 4 end
# '00' -> minute 5 end
# '00' -> minute 6 end
# '00' -> minute 7 end
# '00' -> minute 8 end
# '00' -> minute 9 end
# # Factoid block body end